In [1]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit,compile, execute,IBMQ
from qiskit import BasicAer
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.ibmq import least_busy
from math import pi
from qiskit.mapper._layout import Layout as Ibmlayout
from qiskit.tools.compiler import compile as qcompile
import matplotlib.pyplot as plt
import sys
from qiskit.tools.visualization import circuit_drawer
from parseRealization import *
%matplotlib inline
testDir="./tests/"

C:\Users\krylo\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


In [2]:
#select a backend
IBMQ.load_accounts()
backends = IBMQ.backends(simulator=False,filters=lambda x: x.configuration().n_qubits > 4)
print(backends)
#print(available)
#least_busy = BasicAer.get_backend('qasm_simulator')
least_busy = IBMQ.get_backend('ibmq_16_melbourne')
couplingMap = least_busy.configuration().coupling_map
print(couplingMap,"its length:",len(couplingMap))
qubitsSize = least_busy.configuration().n_qubits


[<IBMQBackend('ibmqx4') from IBMQ()>, <IBMQBackend('ibmq_16_melbourne') from IBMQ()>]
[[1, 0], [1, 2], [2, 3], [4, 3], [4, 10], [5, 4], [5, 6], [5, 9], [6, 8], [7, 8], [9, 8], [9, 10], [11, 3], [11, 10], [11, 12], [12, 2], [13, 1], [13, 12]] its length: 18


In [3]:
def prepareIBMQLayout(qReg,layout):
    ibmLayout = {}
    for elem in layout:
       # print("Element in layout is",elem," physical is",layout[elem])
        quantum = ord(elem)-ord("a")
        physical = ord(layout[elem])-ord("a")
        quantumTuple = (qReg.name, quantum)
        ibmLayout[quantumTuple]=(qReg.name,physical)
    print("tempDictionary",ibmLayout)
    print("items are",ibmLayout.items())
    print("IBMLAYOUT")
    print(ibmLayout)
    return ibmLayout

def bigFunction(fileName):
    ioClass = readCircuitInformation(fileName) 
    answers = ioClass.getKmap()
    size = ioClass.getSize()
    ioClass.readGatesFromFile(fileName)
    backend_sim = BasicAer.get_backend('qasm_simulator')
    error_count = 0
    
    ctg = CircuitTransitionGraph()
    tempStuff = ctg.transformCoupling(couplingMap)
    print(tempStuff,len(tempStuff))
    ctg.setSize(size)
    epoch = 0
    for i in range(0,1):
        #This part corresponds for part of experiment without minimal or no changes
        while epoch < 2:
            print("Current layout of ctg is:",ctg.layout)
            tempLayout = ctg.layout.copy()
            qr,cr,qc = ioClass.createCircuitAndSetInput(i)
            ibmLayout = prepareIBMQLayout(qr,tempLayout)
            qc,qr = ctg.readGatesFromIOClass(qr,qc, ioClass)
            if epoch==0:
                measureFidelityWithoutChanges(qr,cr,qc)
            ctg = fixTheStuff(ctg)
            #This one needs to comply with changes you did
            #qc,qr = ctg.readFixedGatesFromCtg(qr,qc)
           
            qr,cr,qc = ioClass.createCircuitAndSetInput(i)
            qc,qr = ctg.readFixedGatesFromCtg(qr,qc)
            ibmLayout = prepareIBMQLayout(qr,tempLayout)
            compileToSeeCost(qr,cr,qc,ioClass,ibmLayout,i)
            measureToVerifyOutputWtihChanges(qr,cr,qc,ioClass,ibmLayout,i)
            ctg.layOutQubits()

    
def fixTheStuff(ctg):
    print("Missing connections are",ctg.getMissingConnections())
    #This one to fix the changes... fixthemissingedges connects stuff around. did not test though
    ctg.fixMissingEdges()
    print("FIxing the stuff")
    return ctg


def  compileToSeeCost(qr,cr,qc,ioClass,ibmLayout,i):
    #least_busy = BasicAer.get_backend('qasm_simulator')
    qc.measure(qr,cr)
    qobj = qcompile(qc,least_busy,initial_layout=ibmLayout,pass_manager=None)
    #This line provides print of the compiled circuit qasm
    print("Length of IBM compiled circuit with fixes is:",len(qobj.experiments[0].header.as_dict()["compiled_circuit_qasm"]))
    
    #job = execute(qc,least_busy,shots=200)
    #result = job.result()
    #print(result.get_counts())
    #plot_histogram(result.get_counts())
    
def measureFidelityWithoutChanges(qr,cr,qc):
    #least_busy = BasicAer.get_backend('qasm_simulator')
    qc.measure(qr,cr)
    
    qobj = qcompile(qc,least_busy,initial_layout=None,pass_manager=None)
    #This line provides print of the compiled circuit qasm
    print("Length of IBM compiled circuit is:",len(qobj.experiments[0].header.as_dict()["compiled_circuit_qasm"]))
    
#This needs to be implemented    
def measureToVerifyOutputWtihChanges(qr,cr,qc,ioClass,ibmLayout,i):
    least_busy = BasicAer.get_backend('qasm_simulator')
    qc.measure(qr,cr)
    job = execute(qc,least_busy,initial_layout=None,shots=300)
    result = job.result()
    print(result.get_counts())
    error = ioClass.checkOutputs(result.get_counts(),i)
    if error != 0:
        print("ABORT, ERROR WAS produced")
        raise SystemError
    
    #plot_histogram(result.get_counts())


In [4]:
fileName="ex1"
#if testFromFile(filename) == 0:
fileName = testDir+fileName
bigFunction(fileName)

./tests/ex1.pla
The constants are set to .constants 0-----

The circuit has garbage lines in it or file parsed incorrectly, check ioclass
The circuit has constant inputs in it or file parsed incorrectly, check ioclass
['ab', 'bc', 'cd', 'de', 'ek', 'ef', 'fg', 'fj', 'gi', 'hi', 'ij', 'jk', 'dl', 'kl', 'lm', 'cm', 'bn', 'mn'] 18
Current layout of ctg is: {'a': 'a', 'b': 'b', 'c': 'c', 'd': 'd', 'e': 'e', 'f': 'f'}
Constant field is: 0-----
Length of constants field is: 6
Reached here in preparation of inputs!!!, the inputList is ['0', '0', '0', '0', '0', '1']
tempDictionary {('q0', 0): ('q0', 0), ('q0', 1): ('q0', 1), ('q0', 2): ('q0', 2), ('q0', 3): ('q0', 3), ('q0', 4): ('q0', 4), ('q0', 5): ('q0', 5)}
items are dict_items([(('q0', 0), ('q0', 0)), (('q0', 1), ('q0', 1)), (('q0', 2), ('q0', 2)), (('q0', 3), ('q0', 3)), (('q0', 4), ('q0', 4)), (('q0', 5), ('q0', 5))])
IBMLAYOUT
{('q0', 0): ('q0', 0), ('q0', 1): ('q0', 1), ('q0', 2): ('q0', 2), ('q0', 3): ('q0', 3), ('q0', 4): ('q0', 4),

C:\Users\krylo\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


{'110101': 300}
Recalculated weights are: {'ab': 9, 'bc': 7, 'cd': 5, 'de': 3, 'ef': 1}
[('f', 1), ('e', 4), ('d', 8), ('a', 9), ('c', 12), ('b', 16)]
Initial self.qubitconnectionscount is [('f', 1), ('e', 4), ('d', 8), ('a', 9), ('c', 12), ('b', 16)]
wiggle wiggle
Initial self.qubitconnectionscount is [('f', 1), ('e', 4), ('d', 8), ('a', 9), ('c', 12), ('b', 16)]
SELFLAYOUTOFQUBIT ('b', 16)
Self.qubitconnectionscount= [('f', 1), ('e', 4), ('d', 8), ('a', 9), ('c', 12)]
Placing together b c
Self.qubitconnectionscount= [('f', 1), ('e', 4), ('d', 8), ('a', 9)]
Placing together c a
Self.qubitconnectionscount= [('f', 1), ('e', 4), ('d', 8)]
Placing together a d
Self.qubitconnectionscount= [('f', 1), ('e', 4)]
Placing together d e
Self.qubitconnectionscount= [('f', 1)]
Placing together e f
Final layout is {'b': 'm', 'c': 'c', 'a': 'l', 'd': 'n', 'e': 'b', 'f': 'a'}
Current layout of ctg is: {'b': 'm', 'c': 'c', 'a': 'l', 'd': 'n', 'e': 'b', 'f': 'a'}
Constant field is: 0-----
Length of cons

ValueError: 'l' is not in list